In [ ]:
import json

AI_TOOL = 'chatgpt' # copilot/chatgpt

valid_engram_path = './' + AI_TOOL + '_ngrams.txt' # put here the correct path

# get valid ngram from file
VALID_NGRAMS = [line.strip() for line in open(valid_engram_path, 'r') if line.strip() != '']
print(len(VALID_NGRAMS))

def match_valid_ngrams(ngrams_list):
    matched_ngrams = []
    for elem in ngrams_list:
        if elem in VALID_NGRAMS: matched_ngrams.append(elem)
    return matched_ngrams

def save_data(filename, elements):
    with open(filename, 'w') as file:
        for elem in elements:
            file.write(json.dumps(elem) + '\n')

In [ ]:
# load data
commit_elements = []
pr_elements = []
issue_elements = []

with open('commits_' + AI_TOOL + '.jsonl') as f:
    for line in f.readlines():
        line = line.strip()
        commit_elements.append(line)

with open('issues_' + AI_TOOL + '.jsonl') as f:
    for line in f.readlines():
        line = line.strip()
        issue_elements.append(line)

with open('pull-requests_' + AI_TOOL + '.jsonl') as f:
    for line in f.readlines():
        line = line.strip()
        pr_elements.append(line)

print(f"commits: {len(commit_elements)}")
print(f"prs: {len(pr_elements)}")
print(f"issues: {len(issue_elements)}")

In [ ]:
# get only instances featuring valid ngrams
count = 0

valid_issue_instances = []
valid_commit_instances = []
valid_pr_instances = []

no_ngrams_found_issue = []
no_ngrams_found_commit = []
no_ngrams_found_pr = []

for issue,commit,pr in zip(issue_elements,commit_elements,pr_elements):
    count+=1
    issue = json.loads(issue)
    commit = json.loads(commit)
    pr = json.loads(pr)
    ngrams_issue = issue['ngrams']
    ngrams_commit = commit['ngrams']
    ngrams_pr = pr['ngrams']

    if len(ngrams_issue) == 0: no_ngrams_found_issue.append(issue)
    if len(ngrams_commit) == 0: no_ngrams_found_commit.append(commit)
    if len(ngrams_pr) == 0: no_ngrams_found_pr.append(pr)

    if len(ngrams_issue) == 0 and len(ngrams_commit) == 0 and len(ngrams_pr) == 0: continue

    ngrams_issue = [' '.join(tokens) for tokens in ngrams_issue]
    ngrams_commit = [' '.join(tokens) for tokens in ngrams_commit]
    ngrams_pr = [' '.join(tokens) for tokens in ngrams_pr]

    # issue
    matched_ngrams = match_valid_ngrams(ngrams_issue)
    if len(matched_ngrams) > 0:
        issue['valid_ngrams'] = matched_ngrams
        valid_issue_instances.append(issue)
    
    # commit
    matched_ngrams = match_valid_ngrams(ngrams_commit)
    if len(matched_ngrams) > 0:
        commit['valid_ngrams'] = matched_ngrams
        valid_commit_instances.append(commit)

    # pr
    matched_ngrams = match_valid_ngrams(ngrams_pr)
    if len(matched_ngrams) > 0:
        pr['valid_ngrams'] = matched_ngrams
        valid_pr_instances.append(pr)

print(f"valid commits: {len(valid_commit_instances)}")
print(f"valid issues: {len(valid_issue_instances)}")
print(f"valid prs: {len(valid_pr_instances)}")

save_data('valid_commits_' + AI_TOOL + '.jsonl', valid_commit_instances)
save_data('valid_issues_' + AI_TOOL + '.jsonl', valid_issue_instances)
save_data('valid_prs_' + AI_TOOL + '.jsonl', valid_pr_instances)